# Дообучение Text-to-Text модели

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Знакомимся с данными

In [ ]:
df = pd.read_parquet('/content/dataset.parquet')

In [ ]:
df.head(10)

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama fami%ly tree.%,Find information on President Barack Obama\'s ...,2,F0001,"%Fin%%d t%he TIME m%agazin%e photo essay% ""%Ba...",Q00384,"ARE' YOU INTERESTED, ?IN SEEING BARACK OBAMA.S...","yes am interested in' obama,?s family"
1,1,Tell me about %Obama f%amily tree.,Find information on President Barack Obama\'s ...,2,F0001,"F%ind the TIME %magazine photo essay ""Barac%k%...",Q03442,"would ,you like,'' t!o know barac.k 'obamas ge...","y?es i 'wa!nt to kn,ow who made' up his fa'm.ily?"
2,1,Tell me ab%out Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magaz%ine photo ess%ay ""Barac%%k...",Q03402,"wouL,d You LikE To KNOw A!Bout Ob.aMaS An,CESToRs",YES THIS I!S WHAT AM L.OOKIN?G F'OR
3,1,Tell me about Obama family tr%ee.,Find information on President Barack Obama\'s ...,2,F0001,"Find the T%IME magazine photo ess%ay ""Barack O...",Q03725,"wou,ld you like ,to know who is currently aliv...",!N?O FIND THE !BARACK' OB'AMAS FAMILY T?REE TI...
4,1,Tell% me about Obama %family %tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME mag%azine phot%o essay ""Barack O...",Q00769,"ARE ,YOU LOOKING FOR BIOLOGIC.AL INFORMAT..ION...","no! i n!eed the spe,cifi.c time, maga'zine art..."
5,1,Tell me about Obama fam%ily tree.,Find information on President Barack Obama\'s ...,2,F0001,"Fin%d the TIME maga%zine photo%% essay ""Barack...",Q01286,ARE! YOU REFERRIN!G TO THE TIME MAGAZINE ESSAY,"yes please locate ,!the ,photo ,!ess,ay"
6,1,Tell me about Obama family t%ree.,Find information on President Barack Obama\'s ...,2,F0001,"Find th%e %TIME ma%g%azine photo essay ""%Barac...",Q02152,"do you want to? k?now more abou,t ,obamas pa'r...",Yes
7,1,Tell me about Obama family %tree.,Find information on President Barack Obama\'s ...,2,F0001,"Fi%nd the %T%IME magazine p%%ho%to essa%y ""Bar...",Q02527,"hOw maNY gENerAtiONs woulD. YoU likE' ,mE To L...",NONE
8,1,Tell me about %Obama fami%%ly tree.,Find information on President Barack Obama\'s ...,2,F0001,"Fi%nd% the TIM%E mag%azi%ne photo %essay ""Bara...",Q02950,"what would you? like to !kn'ow about oba.m,as ...",".No i ,AM LO!OkING FOR the TimE, MAGAZINe ArtI..."
9,1,Tell me about Obama family tre%e.,Find information on President Barack Obama\'s ...,2,F0001,"F%ind the TIME %magazin%e photo e%ss%ay ""Ba%ra...",Q03023,"whi'ch .f,amily mem.bers. of ob!am!a are you i...",NO !I !JUST WANT TO SEE THE 'BARACK OBAMAS FAM...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11489 entries, 0 to 11488
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   topic_id            11489 non-null  int64 
 1   initial_request     11489 non-null  object
 2   topic_desc          11489 non-null  object
 3   clarification_need  11489 non-null  int64 
 4   facet_id            11489 non-null  object
 5   facet_desc          11489 non-null  object
 6   question_id         11489 non-null  object
 7   question            10727 non-null  object
 8   answer              10727 non-null  object
dtypes: int64(2), object(7)
memory usage: 807.9+ KB


На первый взгляд мы видим необработанные дынные. В некоторых столбцах необходимо убрать лишние знаки и привести к единому регистру. Также наблюдаются пропуски в столбцах вопрос и ответ, что скорее всего связано с ненужностью прояснять некоторые темы.

## Проводим Data Cleaning

Очистим строки от лишних символов и приведем к одному регистру.

In [ ]:
def clean(name):
  df[name] = df[name].str.replace(".", "") \
    .str.replace(",", "") \
    .str.replace("'", "") \
    .str.replace("%", "") \
    .str.replace("!", "") \
    .str.replace("?", "") \
    .str.replace('"', '') \
    .str.replace('\\', '') \
    .str.replace('-', ' ')
  df[name] = df[name].str.lower()

In [ ]:
titles =['initial_request', 'topic_desc', 'facet_desc', 'question', 'answer']
for i in titles:
  clean(i)
df['question'] = df['question'] +'?'
df.head()

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q00384,are you interested in seeing barack obamas fam...,yes am interested in obamas family
1,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03442,would you like to know barack obamas geneology?,yes i want to know who made up his family
2,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03402,would you like to know about obamas ancestors?,yes this is what am looking for
3,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03725,would you like to know who is currently alive ...,no find the barack obamas family tree time mag...
4,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q00769,are you looking for biological information on ...,no i need the specific time magazine article t...


In [ ]:
df.isna().sum()

topic_id                0
initial_request         0
topic_desc              0
clarification_need      0
facet_id                0
facet_desc              0
question_id             0
question              762
answer                762
dtype: int64

Уберем строки с пустыми вопросами и ответами, где метка равна 4, тк в таком случае не обойтись без уточняющего вопроса.

In [ ]:
df = df.drop(df[(df['clarification_need'] == 4) & (df['question'].isna()) & (df['answer'].isna())].index)
df[df['question'].isna()]
df.reset_index(drop= True , inplace= True )
df.head()

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q00384,are you interested in seeing barack obamas fam...,yes am interested in obamas family
1,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03442,would you like to know barack obamas geneology?,yes i want to know who made up his family
2,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03402,would you like to know about obamas ancestors?,yes this is what am looking for
3,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q03725,would you like to know who is currently alive ...,no find the barack obamas family tree time mag...
4,1,tell me about obama family tree,find information on president barack obamas fa...,2,F0001,find the time magazine photo essay barack obam...,Q00769,are you looking for biological information on ...,no i need the specific time magazine article t...


In [ ]:
df.isna().sum()

topic_id                0
initial_request         0
topic_desc              0
clarification_need      0
facet_id                0
facet_desc              0
question_id             0
question              631
answer                631
dtype: int64

Остальные пропуски оставляем, тк с метками 1,2,3 можно обойтись без уточняющих вопросов.

Проверим дубликаты по столбцам 'initial_request', 'topic_desc', 'facet_desc', 'question' и удалим их.

In [ ]:
df.duplicated(['initial_request', 'topic_desc', 'facet_desc', 'question']).sum()

22

In [ ]:
df = df.drop_duplicates(['initial_request', 'topic_desc', 'facet_desc', 'question'])

Проверка удаления

In [ ]:
df.duplicated(['initial_request', 'topic_desc', 'facet_desc', 'question']).sum()

0

In [ ]:
df.nunique()

topic_id               237
initial_request        237
topic_desc             237
clarification_need       4
facet_id               801
facet_desc             801
question_id           3034
question              3033
answer                8732
dtype: int64

Таким образом, мы привели данные в удобный для работы вид, посмотрели наличие пропусков и удалили ненужное. Также удалили дубликаты и посмотрели количество уникальных значений.

## Приводим данные к виду, на котором можно дообучать модель (выдвигаем гипотезы, на счет формата данных для модели)

Думаю, что лишние знаки в изначальных данных не несут за собой какое-то обозначение. Возможно это знаки возникшие при переформатировании данных или попавшие под случайные изменения на сервисах. Регистр тоже скорее всего не имеет никакой роли. Что касается знаков препинания в конце, то для обучения они могут быть важны только в вопросах, поэтому в данном случае мы добавили их.

## Категоризуем/группируем данные, если это возможно

Сгруппируем данные по запросу и описанию темы, так как это кажется самым логичным вариантом.

In [ ]:
df.groupby(['initial_request', 'topic_desc']).agg({'topic_id': 'count'}).sort_values(by='topic_id', ascending=False)

,,topic_id
initial_request,topic_desc,
tell me about avp,find information about events sponsored by avp the association of volleyball professionals,102
how to change the toilet in the house,find information on buying installing and repairing toilets,102
what are the best recepies for grilling,find kabob recipes,102
how to prepare for the gmat,im looking for information to help me prepare for the gmat exam,96
im looking for information on dieting,find reasonable dieting advice that is not fads or medications but reasonable methods for weight loss,96
...,...,...
tell me about making chicken soup from scratch,find recipes for making chicken soup,9
what are treatments for carotid cavernous fistula,what treatments are available for ccft carotid cavernous fistula,9
im interested in history of the electronic medical record,find information on how the electronic medical record (or electronic health record) has evolved through the years,8


Таким образом, мы сгрупировали данные по запросу и описанию темы.

## Оценка достаточности данных для дообучения модели

In [ ]:
df.initial_request.value_counts().to_frame()

,count
initial_request,
tell me about avp,102
what are the best recepies for grilling,102
how to change the toilet in the house,102
how to prepare for the gmat,96
im looking for information on dieting,96
...,...
tell me about making chicken soup from scratch,9
what are treatments for carotid cavernous fistula,9
take me to the current,8


In [ ]:
subject_proportion = df.initial_request.value_counts(normalize=True).mul(100).round(2).astype('str').to_frame() + '%'
subject_proportion

,proportion
initial_request,
tell me about avp,0.9%
what are the best recepies for grilling,0.9%
how to change the toilet in the house,0.9%
how to prepare for the gmat,0.85%
im looking for information on dieting,0.85%
...,...
tell me about making chicken soup from scratch,0.08%
what are treatments for carotid cavernous fistula,0.08%
take me to the current,0.07%


In [ ]:
subject_proportion.shape

(237, 1)

In [ ]:
result_dataset_df = pd.DataFrame()

n=20

for initial_reques in df.initial_request.unique():
  if len(df[df.initial_request == initial_reques]) >= n:
    temp_df = df[df.initial_request == initial_reques].sample(n)
    result_dataset_df = pd.concat([result_dataset_df, temp_df], ignore_index=True)

In [ ]:
result_dataset_df.initial_request.value_counts().to_frame()

,count
initial_request,
tell me about obama family tree,20
tell me more about satilies,20
tell me about madam cj walker,20
child support in indiana,20
tell me about the sit and reach test,20
...,...
im looking for information on rick warren,20
what is yahoo,20
how can i lower my heart rate,20


Нам необходимо было выбрать число повторений тем, для обучения. Наименьшее число повторений в изначальной таблице состоит 5, а наибольшее 102. Чтобы не убирать слишком много тем, но в то же время, чтобы их повторений хватило для обучения, оставим по 20.

## Формируем обучающую (80%) и валидационную выборки

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.reset_index(drop= True , inplace= True )
val_df.reset_index(drop= True , inplace= True )

Данный подход использует функцию train_test_split из библиотеки scikit-learn для разделения исходного набора данных df на две части: обучающую выборку (train_df) и валидационную выборку (val_df).

Аргументы функции:

*df*: исходный набор данных, который нужно разделить.

*test_size=0.2*: параметр, определяющий размер валидационной выборки как 20% от исходных данных, а оставшиеся 80% используются для обучения модели.

*random_state=42*: параметр, обеспечивающий воспроизводимость результатов при каждом запуске кода. Установка random_state в определенное число (в данном случае 42) гарантирует, что разделение данных будет одинаковым при каждом запуске программы.

Этот подход является распространенным при построении и оценке моделей машинного обучения. Разделение данных на обучающую и валидационную выборки позволяет оценить качество модели на отдельном наборе данных, который не участвовал в обучении, что помогает выявить переобучение или недообучение модели.

Посмотрим обучающую выборку

In [ ]:
train_df

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,50,info on dog heat,what is the effect of excessive heat on dogs,2,F0856,what are symptoms of heat stroke and other hea...,Q01041,are you looking for symptoms of dog heat?,yes what happens when they get too hot
1,171,tell me about ron howard,what movies has ron howard directed,2,F0298,find biographical information on director ron ...,Q01116,are you looking for upcoming movies by ron how...,no i was his biography information
2,71,find information about living in india,find information about living in india,3,F0949,find information about buying real estate in i...,Q00586,are you intested in statistics on expats resid...,no i want to buy real estate
3,12,how to hire disc jockey,im looking to hire a disc jockey,2,F0088,what jobs are available for disc jockeys,Q00991,are you looking for reviews of djs?,no i am looking for jobs
4,168,what are the causes of lipoma tumors,what are the causes of lipoma tumors in humans,2,F0278,what are the causes of lipoma tumors in humans,Q00954,are you looking for nearby medical practices t...,no i am looking for what causes them in humans
...,...,...,...,...,...,...,...,...,...
9063,83,tell me about memory,find information about human memory,4,F0998,find information about human memory,Q00831,are you looking for famous people who have gre...,no i am looking for information about human me...
9064,57,tell me about ct jobs,find information about jobs in connecticut,2,F0887,find sites for finding a job in connecticut,Q02835,what kind of trade are you looking for?,i am looking for a site for finding a job in c...
9065,6,tell me more on kansas city railroad,find information on the kansas city southern r...,2,F0900,find information on kcs energy inc and their m...,Q00672,are you looking for a specific energy related ...,no i need merger information about the merger ...
9066,124,tell me about bowflex power pro,find information about the bowflex power pro,2,F0106,find a retailer from whom i can buy a bowflex ...,Q03705,would you like to know where to buy one?,yes i would


Посмотрим валидационную выборку

In [ ]:
val_df

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,140,information about east ridge high school,demographics of east ridge high school in lick...,3,F0166,description of the sports facilities at east r...,Q01689,do you need the address of east ridge high sch...,no i am interested in knowing about the sports...
1,116,what is california franchise tax board,find information about the california franchis...,2,F0071,find information about the california franchis...,Q01835,do you want information on the tax board?,yes i do
2,81,tell me about afganistan,find information about afghanistans history go...,3,F0994,find maps of afghanistan,Q00001,NaN,None
3,49,im looking for flame designs,i am looking for flame graphics to use on a we...,3,F0850,i want to find flame design decals i can put o...,Q00833,are you looking for flame design outline?,it could be an outline as long as it is a deca...
4,97,im looking for information about south africa,find information about the history culture and...,2,F1057,find pages that give the history of south africa,Q02327,do you want to know where south africa is?,no i want to learn about south african history
...,...,...,...,...,...,...,...,...,...
2263,110,where to go if i made a crime in brazil,what are the boundaries of the political juris...,3,F0051,i am looking for information about taking a va...,Q01774,do you want a map of the roads in brazil?,i want to take a vaction there
2264,99,tell me more about satilies,find background information about man made sat...,3,F1067,find providers of satellite television,Q00678,are you looking for a specific kind of maps an...,no this is not related to my search
2265,199,tell me about fybromyalgia,what are the primary symptoms or signs of fybr...,2,F0407,what support groups are there for fybromyalgia...,Q02624,is there something you need to know about this...,i want to know what support groups there are f...
2266,69,find sewing instructions for me,find beginners instructions to sewing both by ...,3,F0942,find materials for teaching sewing to children,Q02515,how experienced are you at sewing?,this is not related to my search


Так, мы сформировали обучающую (80%) и валидационную выборки.

## Общий вывод

Таким образом, мы изчучили данные для дообучения Text-to-Text модели. Далее мы очистили данные и привели их к удобному для работы виду. Для этого мы привели все к одному регистру и убрали лишние знаки, при этом сохранив знак вопроса в конце столбца вопрос. Далее мы убрали пропуски в столбцах вопрос и ответ только при условии, что они не относятся к 4 метке, тк тогда они не обязательны. Также убрали дубликаты. Затем мы сгруппировали данные по запросу и описанию темы. После провели оценку достаточности данных для дообучения модели, при этом решив, что для обучения нам хватит 20 повторений тем. Также мы сформировали обучающую (80%) и валидационную выборки.